# Deploy Infrastructure

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [5]:
import os, sys, json
sys.path.insert(1, '../shared')  # add the shared directory to the Python path
import utils


deployment_name = 'ai-foundry'
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
resource_group_location = "eastus"#"northcentralus" # all the resources will be deployed in this location

build = 0 

# https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/concepts/models
models_config = [
    # {"name": "DeepSeek-R1", "publisher": "DeepSeek", "version": "1", "sku": "GlobalStandard", "capacity": 1},
    # {"name": "Phi-4", "publisher": "Microsoft", "version": "3", "sku": "GlobalStandard", "capacity": 1},
    {"name": "gpt-4o", "publisher": "OpenAI", "version": "2024-05-13", "sku": "GlobalStandard", "capacity": 100},
    {"name": "text-embedding-3-large", "publisher": "OpenAI", "version": "1", "sku": "GlobalStandard", "capacity": 1},
    {"name": "gpt-4.1", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 150},
]

searchservice_sku = "standard"
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "./src/weather/mcp-server"

principalId = ""
if not principalId:
    principalId = !az ad signed-in-user show --query id -o tsv
    principalId = principalId[0]
    
utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 11:52:53.897871 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [6]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 11:52:56.071409 :2s]
👉🏽 Current user: pablocastao@microsoft.com
👉🏽 Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
👉🏽 Subscription ID: 06d043e2-5a2e-46bf-bf48-fffee525f377


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [10]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "modelsConfig": { "value": models_config },
        "principalId": { "value": principalId }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-ai-foundry 
👉🏽 Using existing resource group 'lab-ai-foundry'
⚙️ Running: az deployment group create --name ai-foundry --resource-group lab-ai-foundry --template-file main.bicep --parameters params.json 
⛔ Deployment 'ai-foundry' failed ⌚ 12:01:03.939066 :23s] WARNING: A new Bicep release is available: v0.35.1. Upgrade now by running "az bicep upgrade".
c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\1-infra\modules\standard-dependent-resources.bicep(265,31) : Warning outputs-should-not-contain-secrets: Outputs should not contain secrets. Found possible secret: function 'listKeys' [https://aka.ms/bicep/linter/outputs-should-not-contain-secrets]
c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\1-infra\main.bicep(245,99) : Warning no-hardcoded-env-urls: Environment URLs should not be hardcoded. Use the environment() function to ensure compatibility across 

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [8]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    project = utils.get_deployment_output(output, 'projectConnectionString', 'Project Connection String')    
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    weather_containerapp_url = utils.get_deployment_output(output, 'weatherMCPServerContainerAppFQDN', 'Weather Container App URL')
    bingConnectionName = utils.get_deployment_output(output, 'bingConnectionName', 'Bing Connection Name')
    generative_model_deployment_name = utils.get_deployment_output(output, 'generativeModelDeploymentName', 'Model Deployment Name')
    embedding_model_deployment_name = utils.get_deployment_output(output, 'embeddingModelDeploymentName', 'Embedding Model Name')
    azure_search_endpoint = utils.get_deployment_output(output, 'aiSearchEndpoint', 'Azure Search Endpoint')
    azure_openai_endpoint = utils.get_deployment_output(output, 'openAIEndpoint', 'Azure OpenAI Endpoint')
    azure_storage_account_url = utils.get_deployment_output(output, 'azureStorageAccountUrl', 'Azure Storage Account URL')
    storage_account_name = utils.get_deployment_output(output, 'azureStorageAccountName', 'Azure Storage Account Name')
    azure_ai_services_endpoint = utils.get_deployment_output(output, 'aiServicesEndpoint', 'AI Services Endpoint')

   # Write the project and outputs to an .env file
    with open('../.env', 'w') as env_file:
        env_file.write(f"PROJECT_CONNECTION_STRING='{project}'\n")
        env_file.write(f"APPLICATION_INSIGHTS_NAME='{app_insights_name}'\n")
        env_file.write(f"CONTAINER_REGISTRY_NAME='{container_registry_name}'\n")
        env_file.write(f"WEATHER_CONTAINERAPP_RESOURCE_NAME='{weather_containerapp_resource_name}'\n")
        env_file.write(f"WEATHER_CONTAINERAPP_URL='{weather_containerapp_url}'\n")
        env_file.write(f"BING_CONNECTION_NAME='{bingConnectionName}'\n")
        env_file.write(f"GENERATIVE_DEPLOYMENT_NAME='{generative_model_deployment_name }'\n")
        env_file.write(f"EMBEDDING_DEPLOYMENT_NAME='{ embedding_model_deployment_name }'\n")
        env_file.write(f"AOAI_API_VERSION='2024-12-01-preview'\n")
        env_file.write(f"AZURE_SEARCH_ENDPOINT='{azure_search_endpoint}'\n")
        env_file.write(f"AZURE_OPENAI_ENDPOINT='{azure_openai_endpoint}'\n")       
        env_file.write(f"AZURE_STORAGE_ACCOUNT_URL='{azure_storage_account_url}'\n")
        env_file.write(f"AZURE_SUBSCRIPTION_ID='{subscription_id}'\n")
        env_file.write(f"AZURE_RESOURCE_GROUP='{resource_group_name}'\n")
        env_file.write(f"AZURE_STORAGE_ACCOUNT_NAME='{storage_account_name}'\n")
        env_file.write(f"AZURE_AI_SERVICES_ENDPOINT='{azure_ai_services_endpoint}'\n")


⚙️ Running: az deployment group show --name ai-foundry -g lab-ai-foundry 
⛔ Failed to retrieve deployment: ai-foundry ⌚ 11:53:15.980098 :3s] ERROR: (DeploymentNotFound) Deployment 'ai-foundry' could not be found.
Code: DeploymentNotFound
Message: Deployment 'ai-foundry' could not be found.



### 4️⃣ Build and deploy the MCP Servers

In [6]:
build = build + 1 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

⚙️ Running: az acr build --image weather-mcp-server:v0.1 --resource-group lab-ai-foundry --registry agentcontainerregistryjrpn --file ./src/weather/mcp-server/Dockerfile ./src/weather/mcp-server/. --no-logs 
✅ Weather MCP Server image was successfully built ⌚ 16:30:53.145536 :7s]
⚙️ Running: az containerapp update -n aca-weather-jrpn -g lab-ai-foundry --image "agentcontainerregistryjrpn.azurecr.io/weather-mcp-server:v0.1" 
✅ Weather MCP Server deployment succeeded ⌚ 16:31:21.831264 :28s]
